In [1]:
import csv
import os
import imgaug as ia
ia.seed(1)
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa 
import imageio
import numpy as np

In [2]:
train_csv = 'train.csv'
dir = os.path.join('D:\cats_dogs_dataset')
dir_train = os.path.join(dir, 'train')

In [3]:
def bounding_boxes_to_df(bounding_boxes_object):
    bounding_boxes_array = bounding_boxes_object.to_xyxy_array()
    xmin, ymin, xmax, ymax = bounding_boxes_array[0][0],bounding_boxes_array[0][1],bounding_boxes_array[0][2],bounding_boxes_array[0][3]
    return xmin, ymin, xmax, ymax

In [4]:
image_augmentations = iaa.SomeOf( 2,
    [                                 
    iaa.Affine(scale=(0.5, 1.5)),

    iaa.Affine(rotate=(-60, 60)),

    iaa.Affine(translate_percent={"x":(-0.3, 0.3),"y":(-0.3, 0.3)}),

    iaa.Fliplr(1),

    iaa.Multiply((0.5, 1.5)),

    iaa.GaussianBlur(sigma=(1.0, 3.0)),
    
    iaa.AdditiveGaussianNoise(scale=(0.03*255, 0.05*255))

])

In [5]:
multiple = 3

In [6]:
if not os.path.exists('aug_images_path'):
    os.mkdir('aug_images_path')

with open('aug.csv','w', newline='') as aug:
    writer = csv.writer(aug, delimiter=",")
    for i in range(multiple):
        with open(train_csv, 'r', newline='') as r:
            reader = csv.reader(r, delimiter=",")
            image_postfix = str(i)
            for row in reader:
                aug_row = ()
                path, height, width, x0, y0, x1, y1, class_name, class_id = row
                filename = os.path.basename(path[:-4])

                augmented_path = os.path.join('aug_images_path', filename)+image_postfix+'.jpg'
                image = imageio.imread(path)

                bounding_box_array = [[x0, y0, x1, y1]]

                bounding_box = BoundingBoxesOnImage.from_xyxy_array(bounding_box_array, shape=image.shape)

                image_aug, bounding_box_aug = image_augmentations(image=image, bounding_boxes=bounding_box)

                bounding_box_aug = bounding_box_aug.remove_out_of_image()

                bounding_box_aug = bounding_box_aug.clip_out_of_image()
                
                try:
                    x0, y0, x1, y1 = bounding_boxes_to_df(bounding_box_aug)
                except IndexError:
                    print('box error', filename )
                    continue
                

                imageio.imwrite(augmented_path, image_aug)

                


                aug_row=augmented_path, image_aug.shape[0], image_aug.shape[1], x0, y0, x1, y1, class_name, class_id
                writer.writerow(aug_row)
                if i == 0:
                    writer.writerow(row)

box error Egyptian_Mau_100


C:\Users\79287\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19660800 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\79287\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 787528 bytes but only got 953. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\79287\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 8. 
  warnings.warn(str(msg))


box error leonberger_123
box error British_Shorthair_125
box error Siamese_186
